In [2]:
import os
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

redlight_loc = pd.read_csv('red-light-camera-locations.csv')
redlight_violations = pd.read_csv('red-light-camera-violations.csv')

speed_loc = pd.read_csv('speed-camera-locations.csv')
speed_violations = pd.read_csv('speed-camera-violations.csv')

In [6]:
# find top 3 intersections that have the most red light camera violations
redlight_violations['INTERSECTION'].value_counts()

CALIFORNIA AND DIVERSEY          4593
KOSTNER AND NORTH                4073
BELMONT AND KEDZIE               3899
STONEY ISLAND AND 79TH           3367
ROOSEVELT AND HALSTED            3161
VAN BUREN AND WESTERN            3155
LAKE SHORE DR AND BELMONT        3149
ASHLAND AND FULLERTON            3148
HOLLYWOOD AND SHERIDAN           3131
STATE AND 79TH                   3128
WENTWORTH AND GARFIELD           3128
LAFAYETTE AND 87TH               3111
DIVISION AND DAMEN               3100
CORTLAND AND ASHLAND             3094
ARCHER AND CICERO                3091
OGDEN AND KOSTNER                3089
99TH AND HALSTED                 3088
75TH AND STATE                   3085
55TH AND WESTERN                 3080
STONEY ISLAND AND 76TH           3080
STONY ISLAND/CORNELL AND 67TH    3076
63RD AND STATE                   3066
PETERSON AND WESTERN             3064
HALSTED AND DIVISION             3063
87TH AND VINCENNES               3057
CHICAGO AND CLARK                3055
COLUMBUS AND

In [8]:
# data sets containing subsets of top 3 intersections with the most red light violations
california_diversey_redlight = redlight_violations.loc[redlight_violations['INTERSECTION'] == 'CALIFORNIA AND DIVERSEY']
kostner_north_redlight = redlight_violations.loc[redlight_violations['INTERSECTION'] == 'KOSTNER AND NORTH']
belmont_kedzie_redlight = redlight_violations.loc[redlight_violations['INTERSECTION'] == 'BELMONT AND KEDZIE']